# Clasificador de emails de Thunderbird a .csv

## Objetivo
Leer los mensajes diarios de `yum update` de las máquinas, y en base a eso crear una lista en un archivo .csv, con un "ok" si no ha habido problemas con la actualización o una x en caso de haberlos o no recibir correo.

## Organización
El programa va a tener dos partes claramente diferenciadas:
1. Análisis de los mensajes
2. Escritura del resultado en un archivo .csv

Para más información, leer el README

## 0. Preparación

Para la tarea que tenemos entre manos necesitamos el módulo pyexcel. Pero hemos de definir los objetos correspondientes al problema entre manos, a saber: número de máquinas, lista con sus nombres (ordenada), lista inicializada con "x", y mensajes de no-error.

In [15]:
#Inputs (en línea de comandos en caso de script.py)
sysargv1 = "19" #día
sysargv2 = "01" #mes
sysargv3 = "--show"

num_to_month = {"01": "Enero", "02": "Febrero", "03": "Marzo", "04": "Abril", "05": "Mayo", "06": "Junio", "07": "Julio", "08": "Agosto", "09": "Septiembre", "10": "Octubre", "11": "Noviembre", "12": "Diciembre" }
month = num_to_month[sysargv2]
todaycolumn = int(sysargv1) #la columna es realmente la sysargv1 + 1 (la primera contiene los servidores). Pero como Python empieza en 0 las listas... 

In [16]:
okkeys = ["No se han seleccionando paquetes para ser actualizados","No packages marked for update", "¡Listo!", "Complete!"] #el primer mensaje de error tiene una errata, sí.

In [17]:
import csv
import pyexcel as pe
import numpy as np

In [20]:
xlsx_file = "/home/goznalo/CCC/YumLogwatch/TareasFluor/Updates-2021.ods"
sheet = pe.get_book(file_name=xlsx_file)[month]
arrayeet = sheet.get_array()

In [21]:
numservers = list(np.transpose(arrayeet)[0]).index("zeus.ccc.uam.es")

In [22]:
listservers = []
for i in range(1,numservers + 1): #quitar las filas extra añadidas.
    listservers.append(arrayeet[i][0].lower())

numservers = listservers.index("zeus.ccc.uam.es") + 1

okxlist = ['x'] * numservers

listservers = listservers[0:numservers]

#asegurarse de haber escogido bien los servidores:
if not listservers[0] == "abedul.ccc.uam.es" or not listservers[-1] == "zeus.ccc.uam.es":
    raise ValueError('Algo ha salido mal... la lista de servidores no es correcta.')
else:
    print('okay')

okay


En la lista de máquinas, gran parte de ellas no se cuentan a la hora de revisar estos correos porque no los mandan por defecto. Esto puede deberse a que  estén de baja, que sean sondas o SAIs que no mandan mensajes, etc. Vamos a eliminarlos de la lista de "x"

In [23]:
nomsglist = []
for i in range(1,numservers + 1):
    nomsglist.append(False if not arrayeet[i][1] else True) #coger los espacios en blanco del primer día del mes

numservers = len(listservers)
okxlist = ['x'] * numservers

okxlist =  [a * b for a, b in zip(okxlist,nomsglist)]


# 1. Análisis de los mensajes

Los mensajes se encuentran en un archivo (que Thunderbird considera un directorio de mensajes) dentro de un directorio local creado y usado por Thunderbird.

In [25]:
directorio = "/home/goznalo/.thunderbird/ceesvdh2.default/Mail/fluor.ccc.uam-1.es/YUM2021.sbd/"
archivo = sysargv2 + sysargv1 #modificar con el archivo pertinente, o "prueba1"

path = directorio + archivo

In [26]:
print(path)

/home/goznalo/.thunderbird/ceesvdh2.default/Mail/fluor.ccc.uam-1.es/YUM2021.sbd/0119


Ahora abriremos el archivo (con codificación "iso-8859-1", como podemos ver al ejecutar en una shell `file -i archivo`), que contiene cada uno de los mensajes de esa carpeta, y los separaremos en elementos de una lista, para luego iterar sobre ellos. Para efectuar esta separación, hemos de darnos cuenta de que en cada archivo de mensajes del directorio, los mensajes empiezan con una línea como por ejemplo `From - Wed Jul 29 15:16:48 2020`. Por tanto usaremos como separador `From -`

In [27]:
with open(path, 'r', encoding='iso-8859-1') as file3:
    messages = file3.read().replace('\n', '')

messlist = messages.split("From -")[1:]

Ahora que tenemos los mensajes, hemos de obtener una lista con los remitentes de cada uno de ellos, a la vez que comprobamos si tienen errores o no. Muchas veces hay correos duplicados de las máquinas, con uno de ellos dando error mientras que otro no. En esos casos son errores falsos. Los que son preocupantes son los "reales".

In [45]:
errlist = []
errmessagedict = {}
falseerrcheck = []
for mess in messlist:
    sender = mess.split("<root@")[1].split(">Received:")[0]
    num = listservers.index(sender)
    for oks in okkeys:
        err = 0
        if oks in mess:
            okxlist[num] = "ok"
            falseerrcheck.append(sender)
            break
        else:
            err = 1
    if err == 1:
        errlist.append(sender)
        if "Content-Transfer-Encoding: 8bit" in mess:
            realmess = mess.split("Content-Transfer-Encoding: 8bit")
        elif "X-Cron-Env: <USER=root>" in mess:
            realmess = mess.split("X-Cron-Env: <USER=root>")
        errmessagedict[sender] = realmess[1]

realerrors = list(sorted(set(errlist)-set(falseerrcheck)))
print(okxlist)

['x', 'x', 'x', 'x', '', 'x', 'ok', '', '', '', '', '', 'x', 'ok', 'ok', 'x', 'ok', 'x', 'x', '', '', 'x', 'x', '', '', '', 'x', 'x', 'ok', '', 'x', 'ok', 'ok', 'ok', '', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'x', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'x', '', '', 'x', '', 'x', 'x', 'x', '', 'x', '', 'x', '', 'x', 'x', '', 'x', '', 'x', '', '', '', 'x', 'x', 'x', '', '', '', 'x', '', '', 'x', 'x', 'ok', 'ok', 'ok', '', 'x', '', 'x', '', '', '', 'ok', 'ok', '', 'ok', 'ok', 'ok', '', '', 'x', 'ok', 'ok', 'x', '', 'x', 'x', 'x', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'x', 'x', 'ok', 'ok', 'ok', 'x', 'ok', 'x', 'ok', 'ok', 'ok', 'x', 'ok', 'ok', 'x', 'ok', 'ok', 'ok', 'ok', 'x', 'ok', 'ok', 'ok', 'x', 'x', 'ok', 'ok', 'x', 'ok', 'ok', 'ok', 'x', 'ok

## 2. Escritura del csv y mostrar correos sospechosos

Ya tenemos la lista con los datos que buscábamos. Ahora lo que hemos de hacer es convertirla en un archivo .csv, en formato columna.

In [46]:
with open(f'/home/goznalo/CCC/YumLogwatch/usefulscripts/yum-thunderbird/returns{sysargv2 + sysargv1}.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for val in okxlist:
        writer.writerow([val])

Voilà

In [60]:
errlist = sorted(set(errlist)) #set() para eliminar duplicados

#print(realerrors)
#print(errlist)

centos6err1 = 'Invalid release/repo/arch combination/removing mirrorlist with no valid mirrors: /var/cache/yum/x86_64/6/base/mirrorlist.txt'
centos6err2 = 'Invalid release/repo/arch combination/removing mirrorlist with no valid mirrors: /var/cache/yum/x86_64/6/centosplus/mirrorlist.txt'

####Sacar por pantalla los correos a revisar
print("\033[1mCorreos a revisar:\033[0m")
if len(errlist) < 10 or sysargv3 == "--show":
    for err in errlist:
        if err in realerrors:
            if centos6err1 in errmessagedict[err]:
                print("\t" + str(err) + '\t' + '-!-CENTOS6-!-')
            elif centos6err2 in errmessagedict[err]:
                print("\t" + str(err) + '\t' + '-!-CENTOS6-!-')
            else:
                print("\t" + str(err))
                #print(errmessagedict[err])
        else:
            print("\t" + str(err) + "\t  (falsa alarma, probablemente)")
else:
    print('Demasiados servidores por mostrar (' + str(len(errlist)) + '). Para verlos, repetir el comando con "--show" como tercer argumento')

Correos a revisar:
	ajalvir.ccc.uam.es	-!-CENTOS6-!-
	astato.ccc.uam.es	-!-CENTOS6-!-
	backupx.ccc.uam.es	-!-CENTOS6-!-
	cercedilla.ccc.uam.es	-!-CENTOS6-!-
	cervera.ccc.uam.es	-!-CENTOS6-!-
	cibeles2-006.ccc.uam.es	  (falsa alarma, probablemente)
	cibeles2-172.ccc.uam.es	  (falsa alarma, probablemente)
	cibeles2-337.ccc.uam.es	  (falsa alarma, probablemente)
	hera.ccc.uam.es	-!-CENTOS6-!-
	platino.ccc.uam.es	-!-CENTOS6-!-
	plomo.ccc.uam.es	-!-CENTOS6-!-
